In [1]:
from gensim.models import word2vec
import multiprocessing
from tqdm import tqdm
import pandas as pd
import numpy as np
import multiprocessing
from joblib import Parallel, delayed
from multiprocessing import Pool, cpu_count
import dask.dataframe as dd
from dask.distributed import LocalCluster, Client
import csv
import pandarallel
import gc

### Исходный датасет

2 файла:
- data/user_item_interaction.csv колонки: user, item (song_id)
- data/track_meta_processed.tsv колонки song_id, song_name, band, band_preprocessed, band_id

Для обучения word2vec модели рекомендуется использовать band_preprocessed, там содержатся нормализованные названия исполнителей, так с моделью будет проще работать.

Чтобы обучить модель, на вход ей нужно подать файл в формате:
- один пользователь одна строка
- song_id расположены в хронологическом порядке (генератор их сам подменит на нормализованное название) и разделены табуляцией ('\t')

data/song_lists_preprocessed_small.txt - пример, как будет выглядет итоговый файл, на котором можно обучить модель, на нем можно попробовать обучить модель уже сейчас, он содержет лишь 50000 пользователей, после выполнения домашнего задания у вас будет аналогичный файл с 900000 пользователей и качество модели станет заметно лучше

### В качестве домашнего задания нужно будет написать функцию для преобразование датасета:
- когда весь датасет помещается в памяти компьютера
- когда датасет не помещается в памяти (скажем у вас есть 500МБ оперативной памяти)

Последовательность айтемов должна быть сохранена, последовательность пользовательских строк в файле не важна.

Метрика: время преобразования (чем быстрее, тем лучше)
Для честности все варианты мы отскорим на одинаковых мощностях.

In [2]:
input_path = 'data/user_item_interaction.csv'
output_path = 'data/song_lists_preprocessed_tmp.txt'

meta_path = 'data/track_meta_processed.tsv'

w2v_dataset_path = 'data/song_lists_preprocessed_small.txt'

item_type = 'track_id'

### aashink

In [11]:
%%time
with open(input_path, newline='\n') as read_file:
    file = csv.reader(read_file, delimiter=',')
    with open(output_path, 'w') as file_write:
        header = next(file)
        first_row = next(file)
        user, items_list = first_row[0], [first_row[1]]
        for row in file:
            user_new = row[0]
            if user_new == user:
                items_list.append(row[1])
                user = user_new
            else:
                file_write.write('{}\n'.format('\t'.join(items_list)))
                items_list = [row[1]]
                user = user_new

CPU times: user 1min 2s, sys: 1.11 s, total: 1min 3s
Wall time: 1min 4s


### andreytyu

In [9]:
def transform_w2v(input_path='data/user_item_interaction.csv',
                  output_path='output.txt',
                 first_user_id='0'):
    '''
    Transforms data with vanilla python, cause libraries are overrated
    '''
    previous_user = first_user_id
    
    with open(input_path) as file_in, open(output_path, mode='w') as file_out:
        
            #skip header
            next(file_in)
            
            #ugly way to treat first row
            user,item = next(file_in)[:-1].split(',')
            file_out.write(item)
            
            for line in file_in:
                
                user,item = line[:-1].split(',')
                
                if user == previous_user:
                    file_out.write('\t'+item)
                else:
                    file_out.write('\n'+item)
                    previous_user = user

In [10]:
%%time
transform_w2v(input_path, output_path)

CPU times: user 53.7 s, sys: 1.16 s, total: 54.8 s
Wall time: 55.1 s


### andkhol

In [66]:
def file_transform(input_file=input_path,
                   output_file=output_path,
                   chunksize=None):
    """
    1. Преобразование входного файла input_file, колонки: user, item (band_id)
    в файл output_file в формате: один пользователь - одна строка,
    band_id расположены в хронологическом порядке и разделены табуляцией;
    2. chunksize - указывается размер chunk'а в случае обработки больших файлов;
    3. Последовательность пользовательских строк в выходном файле может не соблюдаться.
    """

    if chunksize == None:
        pd.read_csv(input_file,
                    dtype={
                        'user': np.uint32,
                        'item': str
                    },
                    low_memory=False).groupby('user')['item'].apply(
                        lambda x: '\t'.join(x)).to_csv(output_file,
                                                       index=False,
                                                       sep='\n',
                                                       mode='w',
                                                       header=False)
    else:
        last_user = -1
        no_first = False

        with open(output_file, 'w') as f:
            for chunk in pd.read_csv(input_file,
                                     dtype={
                                         'user': np.uint32,
                                         'item': str
                                     },
                                     low_memory=False,
                                     chunksize=chunksize):

                user_unique = chunk['user'].unique()

                if no_first:
                    if user_unique[0] == last_user:
                        f.write('\t')
                    else:
                        f.write('\n')

                f.write('\n'.join(
                    chunk.groupby('user')['item'].apply(
                        lambda x: '\t'.join(x))))

                last_user = user_unique[-1]
                no_first = True

    return None

In [16]:
%%time

file_transform(input_path)

CPU times: user 2min 33s, sys: 10.5 s, total: 2min 43s
Wall time: 2min 45s


In [67]:
%%time

file_transform(input_path, chunksize=20000)

CPU times: user 2min 15s, sys: 11.6 s, total: 2min 27s
Wall time: 2min 27s


### AzurTheOwl

In [30]:
%%time

df = pd.read_csv(
    input_path,
    dtype={'user': np.int64, 'item': np.str},
)
df.groupby('user')['item'].agg('\t'.join).to_csv(output_path, index=False, header=False)

CPU times: user 1min 3s, sys: 9.04 s, total: 1min 12s
Wall time: 1min 15s


In [68]:
import dask
import dask.distributed
from dask.distributed import Client, LocalCluster
import dask.dataframe as dd
import numpy as np

dask.config.set({'distributed.worker.memory.terminate': False})
dask.config.get('distributed.worker.memory')

cluster = LocalCluster(
    n_workers=1, 
    threads_per_worker=1,
    memory_limit='512MB'
)
client = Client(cluster)


client

Client Scheduler: tcp://127.0.0.1:60931 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 1 Cores: 1 Memory: 512.00 MB


In [69]:
%%time
df = dd.read_csv(
    input_path,
    blocksize='25MB',
    dtype={'user': np.int64, 'item': np.str}
)

join_str = dd.Aggregation('join_str', lambda x0: x0.agg("\t".join), lambda x0: x0.agg("\t".join))
agg_fn = {'item': join_str}

df = df.groupby('user').agg(agg_fn)

df.to_csv(
    output_path,
    index=False,
    header=False,
    single_file=True
)

CPU times: user 8.88 s, sys: 8.8 s, total: 17.7 s
Wall time: 2min 10s


['/Users/da/PycharmProjects/datagym_recsys/w2v/w2v_homework/data/song_lists_preprocessed_tmp.txt']

In [70]:
client.close()
cluster.close()

### BegunovNA

In [13]:
def f(list_of_dfs):
    return ('\t'.join(str(s.item) for s in list_of_dfs.itertuples()) + '\n')

def applyParallel(dfGroup, func):
    Lst_res = Parallel(n_jobs = multiprocessing.cpu_count())(delayed(func)(group) for name, group in dfGroup)
    return Lst_res

In [14]:
%%time

df_res = pd.DataFrame()
df = pd.read_csv(input_path)
patients_table_raw = applyParallel(df.groupby('user'), f)
df_res['item'] = patients_table_raw

df_res['item'].apply(lambda s: s[:-1]).to_csv(output_path, header=None, index=False)

CPU times: user 7min 45s, sys: 21.6 s, total: 8min 7s
Wall time: 8min 9s


In [71]:
def f2(list_of_dfs):
    return ('\t'.join(str(s.item) for s in list_of_dfs.itertuples()))

In [72]:
%%time

res = []
i = 0
for d in pd.read_csv(input_path, chunksize=1000000, iterator=True):
    patients_table_raw_ = applyParallel(d.groupby('user'), f2)
    res_ = d.groupby('user').count()
    res_['item'] = patients_table_raw_
    res.append(res_)
    i+=1

CPU times: user 7min 35s, sys: 17.8 s, total: 7min 53s
Wall time: 7min 47s


### botvinkine

In [69]:
%%time
# читаем для чистоты, т.к. время на чтение будет включаться при оценке
df = pd.read_csv(input_path)
# определим reduce-функцию
def inline_reduce(gr):
    gr = gr.astype(str)
    return '\t'.join(gr.tolist()) # обещают, что tolist() всегда сохраняет порядок
# применим к данным
df.groupby('user')['item'].apply(inline_reduce).to_csv(output_path, index=False, header=False)

CPU times: user 3min 43s, sys: 8.83 s, total: 3min 52s
Wall time: 3min 54s


In [73]:
%%time
# определим reduce-функцию
def inline_reduce(gr):
    gr = gr.astype(str)
    return '\t'.join(gr.tolist())
def stream_proc(path_in, path_out, chunk_size):
    # разбивка на бачи
    chunks = pd.read_csv(path_in, chunksize=chunk_size)
    # последнего всегда будем переносить в следующий бач
    last_user = pd.DataFrame()
    for chunk in chunks:
        # добавим последнего из предыдущего бача
        chunk = pd.concat([last_user, chunk])
        # обновим последнего
        last_id = chunk['user'].iloc[-1]
        is_last = chunk['user']==last_id
        chunk, last_user = chunk[~is_last], chunk[is_last]
        # применим к бачу
        chunk.groupby('user')['item'].apply(inline_reduce).to_csv(path_out, index=False, header=False, mode='a')
    # в последнем баче напрасно отложили последнего пользователя, добавим
    last_user.groupby('user')['item'].apply(inline_reduce).to_csv(path_out, index=False, header=False, mode='a')
stream_proc(input_path, output_path[0:-4]+'_chunk.txt', 20000)

CPU times: user 3min 56s, sys: 9.96 s, total: 4min 6s
Wall time: 4min 5s


### burekat

In [5]:
def applyParallel(dfGrouped, func):
    with Pool(cpu_count()) as p:
        ret_list = p.map(func, [{name: group} for name, group in dfGrouped])
    return ret_list
def items2list(d):
    (key, value), = d.items()
    return {key: '\t'.join(map(str, value.item.tolist()))}

In [7]:
%%time

df = pd.read_csv(input_path, error_bad_lines = False, warn_bad_lines = False)
# df = df.dropna()

r = applyParallel(df.groupby('user'), items2list)
users = [k for x in r for k, v in x.items()]
items = [v for x in r for k, v in x.items()]
result1 = pd.DataFrame({'user': users, 'items': items})

result1['items'].to_csv(output_path, index=False, header=False)

CPU times: user 3min 43s, sys: 24.4 s, total: 4min 7s
Wall time: 4min 20s


In [6]:
%%time
chunksize = 20000000
chunks = []
for chunk in tqdm(pd.read_csv(input_path, error_bad_lines = False, warn_bad_lines = False, chunksize=chunksize)):
    chunk = chunk.dropna()
    chunk_ = applyParallel(chunk.groupby('user'), items2list)
    chunks.append(chunk_)
    
chunksall = [y for x in chunks for y in x]
users = [k for x in chunksall for k, v in x.items()]
items = [v for x in chunksall for k, v in x.items()]
result2_ = pd.DataFrame({'user': users, 'items': items})

# если пользователь попал в 2 чанка, джойним его треки и вклеиваем новый треклист вместо дубликата
duplicated = result2_[result2_.duplicated(subset = 'user', keep = False)]\
                                        .groupby('user')['items'].apply(lambda x: '\t'.join(x))
result2 = pd.concat([result2_.drop_duplicates(subset = 'user', keep = False), pd.DataFrame(duplicated).reset_index()])

5it [03:34, 42.86s/it]


CPU times: user 3min 14s, sys: 16.1 s, total: 3min 30s
Wall time: 3min 36s


### GoryachevaT

In [100]:
%%time
reader = pd.read_table(input_path, chunksize=100000, sep=',')
lst_to_remember = []
prev_max_u = -1

with open(output_path, 'w') as f:
    
    for chunk in reader:
        curr_min_u = chunk['user'].min()
        curr_max_u = chunk['user'].max()
    
        index, counts = np.unique(chunk['user'].values,  return_counts=True)
        tmp = np.split(chunk['item'].values,  np.cumsum(counts)[:-1])
        tmp = np.array(tmp)
        tmp = np.hstack((index.reshape((index.shape[0],  1)),
                 tmp.reshape((tmp.shape[0],  1)),
                ))
        
        if curr_min_u==prev_max_u:
            left = np.hstack((lst_to_remember, tmp[0][1]))
            tmp[0][1] = left
        else:
            f.write('\t'.join(str(item) for item in lst_to_remember) + '\n') 
         
        prev_max_u = tmp[-1][0]
        lst_to_remember = tmp[-1][1]
        
        for pair in tmp[:-1]:
            f.write('\t'.join(str(item) for item in pair[1]) + '\n') 
    
    f.write('\t'.join(str(item) for item in lst_to_remember) + '\n')

CPU times: user 1min 17s, sys: 3.32 s, total: 1min 20s
Wall time: 1min 20s


### VariyaKh

In [10]:
def get_items_list(array):
    return np.split(array[:, 1],
                    np.argwhere(np.diff(array, axis=0)[:, 0]).flatten() + 1)


def write_to_file(fname, items_list):
    with open(fname, 'a') as f:
        for items in items_list:
            np.savetxt(f, [items], delimiter='\t', newline='\n', fmt='%d')
           
        
def convert_w2v_format(src_fname,
                       dst_fname,
                       chunksize=None,
                       overwrite=True):
    
    if overwrite:
        open(dst_fname, 'w').close()
    
    if chunksize is None:
        df = pd.read_csv(src_fname,
                         dtype=np.int32,
                         delimiter=',',
                         engine='c')
        items_list = get_items_list(df.values)
        write_to_file(dst_fname, items_list)
        return True
        
    first_key = None
    last_key = None
    last_items = None

    for chunk in pd.read_csv(src_fname,
                             dtype=np.int32,
                             delimiter=',',
                             engine='c',
                             chunksize=chunksize):
        items_list = get_items_list(chunk.values)
        first_key = chunk.iloc[0, 0]
        if last_items is not None:
            if (last_key == first_key):
                items_list[0] = np.concatenate((last_items, items_list[0]))
            else:
                items_list.insert(0, last_items)
        last_items = items_list.pop()
        last_key = chunk.iloc[-1, 0]
        write_to_file(dst_fname, items_list)
    write_to_file(dst_fname, [last_items])
    return True

In [51]:
%%time
convert_w2v_format(input_path, output_path)

CPU times: user 59.2 s, sys: 2.09 s, total: 1min 1s
Wall time: 1min 1s


True

In [12]:
%%time
convert_w2v_format(input_path, output_path, chunksize=1000000)

CPU times: user 1min 2s, sys: 1.96 s, total: 1min 4s
Wall time: 1min 4s


True

### DmitrievEgor94

In [23]:
def big_ram_trans(input_path, output_path) -> None :
    user_item = pd.read_csv(input_path)

    user_item['item'] = user_item.item.astype(str)
    user_item.groupby('user').item.apply('\t'.join).to_csv(output_path, header=False, index=False)

In [22]:
%%time
big_ram_trans(input_path, output_path)

CPU times: user 2min 13s, sys: 11.6 s, total: 2min 25s
Wall time: 2min 28s


In [16]:
%%time
big_ram_trans(input_path, output_path)

CPU times: user 2min 47s, sys: 22.5 s, total: 3min 10s
Wall time: 3min 17s


In [3]:
def small_ram_trans(input_path, ouput_paths, chunk_size = 1) -> None:
    for gm_chunk in pd.read_csv(input_path, chunksize=chunk_size):
        gm_chunk['item'] = gm_chunk.item.astype(str)
        gm_chunk.groupby('user').item.apply('\t'.join).to_csv(output_path, header=False, index=False, mode='a')
        
        del gm_chunk

In [4]:
%%time

ram_size, row_size = 500, 16/1024**2
chunk_size = (ram_size - 50) // row_size

small_ram_trans(input_path, output_path, chunk_size)

CPU times: user 2min 42s, sys: 7.19 s, total: 2min 49s
Wall time: 2min 50s


### jtuvaleva

In [5]:
def collect_list(df):
    items = df['item'].to_list()
    return '\t'.join(str(item) for item in items)

In [4]:
%%time

df = pd.read_csv(input_path)
df.groupby('user').apply(collect_list).to_csv(output_path, 
             index = False,
             header = False,
             quoting = 0,
             escapechar='"')

CPU times: user 1min 22s, sys: 5.14 s, total: 1min 27s
Wall time: 1min 28s


In [3]:
client = Client(n_workers=1, 
                threads_per_worker=2, processes=False, memory_limit='1GB')
client

Client Scheduler: inproc://172.20.10.9/28021/1 Dashboard: http://172.20.10.9:8787/status,Cluster Workers: 1 Cores: 2 Memory: 1000.00 MB


In [6]:
%%time
df = dd.read_csv(input_path)

df.groupby('user').apply(collect_list).compute().sort_index().to_csv('data/by_dask.txt', 
             index = False,
             header = False,
             quoting = 0,
             escapechar='"')

/Users/da/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:3: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  This is separate from the ipykernel package so we can avoid doing imports until
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 753.09 MB -- Worker memory limit: 1000.00 MB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 793.41 MB -- Worker memory limit: 1000.00 MB
distributed.worker - WARNING - Worker is at 108% memory usage. Pausing worker.  Process memory: 1.08 GB -- Worker memory limit: 1000.00 MB
distributed.worke

distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 888.20 MB -- Worker memory limit: 1000.00 MB
distributed.worker - WARNING - Worker is at 62% memory usage. Resuming worker. Process memory: 626.89 MB -- Worker memory limit: 1000.00 MB
distributed.worker - WARNING - Worker is at 84% memory usage. Pausing worker.  Process memory: 845.92 MB -- Worker memory limit: 1000.00 MB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 765.10 MB -- Worker memory limit: 1000.00 MB
distributed.worker - WARNING - Worker is at 76% memory usage. Resuming worker. Process memory: 765.10 MB -- Worker memory limit: 1000.00 MB
distributed.worker - WARNING - Worker is at 81% memory usage. Pausing worker.  Process memory: 810.31 MB -- Worker memory limit: 1000.00 MB
distributed.worker - WARNING - Memory us

distributed.worker - WARNING - Worker is at 49% memory usage. Resuming worker. Process memory: 497.76 MB -- Worker memory limit: 1000.00 MB
distributed.worker - WARNING - Worker is at 91% memory usage. Pausing worker.  Process memory: 910.91 MB -- Worker memory limit: 1000.00 MB
distributed.worker - WARNING - Worker is at 50% memory usage. Resuming worker. Process memory: 505.25 MB -- Worker memory limit: 1000.00 MB
distributed.worker - WARNING - Worker is at 101% memory usage. Pausing worker.  Process memory: 1.02 GB -- Worker memory limit: 1000.00 MB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 1.07 GB -- Worker memory limit: 1000.00 MB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 939.03 MB -- Worker memory limit: 1000.00 MB
distributed.worker - WARNING - Memory use i

distributed.worker - WARNING - Worker is at 96% memory usage. Pausing worker.  Process memory: 960.14 MB -- Worker memory limit: 1000.00 MB
distributed.worker - WARNING - Worker is at 58% memory usage. Resuming worker. Process memory: 586.69 MB -- Worker memory limit: 1000.00 MB
distributed.worker - WARNING - Worker is at 81% memory usage. Pausing worker.  Process memory: 810.89 MB -- Worker memory limit: 1000.00 MB
distributed.worker - WARNING - Worker is at 57% memory usage. Resuming worker. Process memory: 574.38 MB -- Worker memory limit: 1000.00 MB
distributed.worker - WARNING - Worker is at 95% memory usage. Pausing worker.  Process memory: 952.29 MB -- Worker memory limit: 1000.00 MB
distributed.worker - WARNING - Worker is at 57% memory usage. Resuming worker. Process memory: 572.74 MB -- Worker memory limit: 1000.00 MB
distributed.worker - WARNING - Worker is at 95% memory usage. Pausing worker.  Process memory: 952.25 MB -- Worker memory limit: 1000.00 MB
distributed.worker -

distributed.worker - WARNING - Worker is at 57% memory usage. Resuming worker. Process memory: 573.08 MB -- Worker memory limit: 1000.00 MB
distributed.worker - WARNING - Worker is at 80% memory usage. Pausing worker.  Process memory: 804.31 MB -- Worker memory limit: 1000.00 MB
distributed.worker - WARNING - Worker is at 53% memory usage. Resuming worker. Process memory: 538.54 MB -- Worker memory limit: 1000.00 MB
distributed.worker - WARNING - Worker is at 89% memory usage. Pausing worker.  Process memory: 895.16 MB -- Worker memory limit: 1000.00 MB
distributed.worker - WARNING - Worker is at 44% memory usage. Resuming worker. Process memory: 443.35 MB -- Worker memory limit: 1000.00 MB
distributed.worker - WARNING - Worker is at 83% memory usage. Pausing worker.  Process memory: 833.43 MB -- Worker memory limit: 1000.00 MB
distributed.worker - WARNING - Worker is at 44% memory usage. Resuming worker. Process memory: 443.77 MB -- Worker memory limit: 1000.00 MB
distributed.worker -

...

distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 1.06 GB -- Worker memory limit: 1000.00 MB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 1.06 GB -- Worker memory limit: 1000.00 MB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 1.06 GB -- Worker memory limit: 1000.00 MB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 1.06 GB -- Worker memory limit: 1000.00 MB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 1.05 GB -- Worker memory limit: 1000.00 MB
distribute

distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 1.02 GB -- Worker memory limit: 1000.00 MB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 1.02 GB -- Worker memory limit: 1000.00 MB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 1.02 GB -- Worker memory limit: 1000.00 MB
distributed.worker - WARNING - Worker is at 32% memory usage. Resuming worker. Process memory: 327.11 MB -- Worker memory limit: 1000.00 MB
distributed.worker - WARNING - Worker is at 87% memory usage. Pausing worker.  Process memory: 872.23 MB -- Worker memory limit: 1000.00 MB
distributed.worker - WARNING - Worker is at 33% memory usage. Resuming worker. Process memory: 331.91 MB -- Worker memory limi

distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 1.02 GB -- Worker memory limit: 1000.00 MB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 1.02 GB -- Worker memory limit: 1000.00 MB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 1.02 GB -- Worker memory limit: 1000.00 MB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 1.02 GB -- Worker memory limit: 1000.00 MB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 1.02 GB -- Worker memory limit: 1000.00 MB
distribute

distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 1.01 GB -- Worker memory limit: 1000.00 MB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 1.01 GB -- Worker memory limit: 1000.00 MB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 1.01 GB -- Worker memory limit: 1000.00 MB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 1.01 GB -- Worker memory limit: 1000.00 MB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 1.01 GB -- Worker memory limit: 1000.00 MB
distribute

CPU times: user 4min 8s, sys: 2min 57s, total: 7min 6s
Wall time: 6min 46s


### kodimka

In [16]:
%%time
! sed '1d' data/user_item_interaction.csv > data/user_item_interaction_clear.csv

CPU times: user 249 ms, sys: 73.3 ms, total: 322 ms
Wall time: 26.7 s


In [17]:
input_file = 'data/user_item_interaction_clear.csv'

In [18]:
%%time
%%bash -s "$input_file"
awk -F, '{a[$1]=a[$1]?a[$1]"\t"$2:$2;}END{for (i in a)print i, a[i]|"sort -t',' -nk1 ";}' OFS=, $1 | cut -d"," -f2- > data/song_lists_preprocessed_tmp.txt
# ! awk -F, '{a[$1]=a[$1]?a[$1]"\t"$2:$2;}END{for (i in a)print i," " a[i];}' OFS=, data/user_item_interaction_s.csv | sort -n

CPU times: user 7.42 ms, sys: 9.7 ms, total: 17.1 ms
Wall time: 4min 1s


### msazxc

In [23]:
def preprocessing_full(input_path):
    df = pd.read_csv(input_path)
    df_user = df.groupby(['user']).agg({'item': lambda x: str(list(x)).strip('[]').replace(',', '').replace(' ', '\t')})
    df_user.reset_index(inplace = True)
    result = pd.DataFrame(pd.concat([pd.Series(list(df_user['user']), index = range(0, df_user['user'].shape[0] * 2, 2)), 
    pd.Series(list(df_user['item']), index = range(1, df_user['user'].shape[0] * 2, 2))]).sort_index())
    result.to_csv(output_path, index = None)

In [24]:
%%time
result_full = preprocessing_full(input_path)

CPU times: user 57.4 s, sys: 3.58 s, total: 1min
Wall time: 1min 1s


In [22]:
def preprocessing_chunk(input_path):
    data = pd.read_csv(input_path, chunksize = 10**7)
    dd = pd.DataFrame()
    for df_ in data:
        dd = dd.append([df_.groupby(['user']).agg({'item': 
                                                   lambda x: str(list(x)).strip('[]').replace(',', '').replace(' ', '\t')})])
    dd = dd.reset_index().groupby(['user']).agg({'item': lambda x: (x+'\t').sum()[:-1]}).reset_index()
    dd = pd.DataFrame(pd.concat([pd.Series(list(dd['user']), index = range(0, dd.shape[0] * 2, 2)), 
        pd.Series(list(dd['item']), index = range(1, dd.shape[0] * 2, 2))]).sort_index())
    with open(output_path, 'w') as f:
        f.write(dd)

In [23]:
%%time
result_chunk = preprocessing_chunk(input_path)

CPU times: user 4min 30s, sys: 6.24 s, total: 4min 37s
Wall time: 4min 36s


### RasovArsenii

In [3]:
def f(list_of_dfs):
    return ('\t'.join(str(s.item) for s in list_of_dfs.itertuples()) + '\n')

def f_(list_of_dfs):
    return ('\t'.join(str(s.item) for s in list_of_dfs.itertuples()))

def applyParallel(dfGrouped, func):
    retLst = Parallel(n_jobs=multiprocessing.cpu_count())(delayed(func)(group) for name, group in dfGrouped)
    return retLst

In [30]:
%%time
df = pd.read_csv(input_path)
patients_table_raw = applyParallel(df.groupby('user'), f)
with open(output_path, 'w') as fl:
    fl.write(''.join(patients_table_raw))

CPU times: user 7min 12s, sys: 15.4 s, total: 7min 27s
Wall time: 7min 23s


In [18]:
%%time
res_ = []
i = 0
for d in pd.read_csv(input_path, chunksize=10000000, iterator=True):
#     print(i)
    patients_table_raw_ = applyParallel(d.groupby('user'), f_)
    res__ = d.groupby('user').count()
    res__['item'] = patients_table_raw_
    res_.append(res__)
#     print(res__.info())
    i+=1
    
res = pd.concat(res_).groupby('user').agg(lambda x: '\t'.join(x)) + '\n'
with open(output_path, 'w') as fl:
    fl.write(''.join(res['item'].values))

CPU times: user 7min 14s, sys: 12.5 s, total: 7min 26s
Wall time: 7min 22s


### RAVasiliev

In [38]:
%%time

df = pd.read_csv(input_path)
df['item'] = df['item'].apply(str)
df.set_index('user', inplace=True)
res_df = df.groupby(['user'])['item'].apply(lambda x: "%s" % '\t'.join(x))
pd.DataFrame(res_df.values).reset_index().to_csv(output_path, sep = '\n', index = None)

CPU times: user 2min 18s, sys: 17.5 s, total: 2min 36s
Wall time: 2min 41s


In [47]:
res_df.to_csv(output_path, index=False, header=False)

In [8]:
%%time

ite = 0

for chunk in pd.read_csv(input_path, chunksize=10000):
    ite += 1
    if ite == 1:
        dic = (chunk.groupby('user').item.apply(list).to_dict())
    else:
        dic = (pd.concat([old_slov, chunk]).groupby('user').item.apply(list).to_dict())
        
    #print(set([*dic]) - set([max(dic, key=int)]))
    old_slov = pd.DataFrame([(max(dic, key=int), dic[max(dic, key=int)][i])\
                        for i in range(len(dic[max(dic, key=int)]))])
    old_slov.columns = ['user', 'item']
    
    my_df = pd.DataFrame([(i, dic[i]) for i in set(set([*dic]) - set([max(dic, key=int)]))])
    
    
    if len(my_df) > 0:
        my_df[1] = my_df[1].apply(lambda x: '\t'.join(map(str, x)))

        my_df.to_csv(output_path, mode='a', header = None, index = None, sep = '\n')

dic = (old_slov.groupby('user').item.apply(list).to_dict())
my_df = pd.DataFrame([(i, dic[i]) for i in set([*dic])])
my_df.to_csv(output_path, mode='a',\
                    header = None, index = None, sep = '\n')

CPU times: user 4min 16s, sys: 15.2 s, total: 4min 31s
Wall time: 4min 31s


### maryblack

In [57]:
def f(v):
    return '\t'.join(map(lambda x: str(x),v))

def make_w2v_data(input_path, output_path):    
    df = pd.read_csv(input_path, dtype={'user':'int32', 'item':'int32'})
    
    groups = df.groupby(pd.Grouper('user'))
    grps = groups.groups
    grps.update({k: f(v.values) for k, v in grps.items()})
    pd.DataFrame.from_dict(grps, orient='index').to_csv(output_path, index=False, header=False)

In [58]:
%%time
make_w2v_data(input_path, output_path)

CPU times: user 2min 2s, sys: 6.09 s, total: 2min 8s
Wall time: 2min 11s


In [13]:
def f(v):
    return '\t'.join(map(lambda x: str(x),v))

def make_w2v_optimize_memory(input_path, output_path):
    SIZE = 1000000
    
    for i, c in enumerate(pd.read_csv(input_path, dtype={'user':'int32', 'item':'int32'}, chunksize=SIZE)):  
        groups = c.groupby(pd.Grouper('user'))
        grps = groups.groups
        grps.update({k: list(v.values) for k, v in grps.items()})
        
#         print('сделали дикт')
              
        if i==0:
            
            res = grps

        else:
            for key, value in grps.items():
                res.setdefault(key, []).extend(value)#можно через табуляцию сразу добавлять, быстрее будет
                
#         print('запишем в файл первые len(grps)-1 строк, остальное удалим')
        
        last_key = max(res.keys())
   
        curr_res = {last_key: res[last_key].copy()}
    
        
        to_write = dict(list(res.items())[:len(res) - 1])
        if len(c)<SIZE:#номер последнего chunk
            print('last chunk')
            to_write = res.copy()
        
        with open(output_path, 'a+', newline="") as csv_file:  
            w = csv.writer(csv_file)
            to_write.update({k: f(v) for k, v in to_write.items()})
            for key, value in to_write.items():
                w.writerow([value])
                
        del res, to_write
        gc.collect()
        
        res = curr_res
        
        print(f'шаг {i} сделан')
        
#     res.update({k: f(v) for k, v in res.items()})
#     pd.DataFrame.from_

In [16]:
%%time
make_w2v_optimize_memory(input_path, output_path+'_chunks')

шаг 0 сделан
шаг 1 сделан
шаг 2 сделан
шаг 3 сделан
шаг 4 сделан
шаг 5 сделан
шаг 6 сделан
шаг 7 сделан
шаг 8 сделан
шаг 9 сделан
шаг 10 сделан
шаг 11 сделан
шаг 12 сделан
шаг 13 сделан
шаг 14 сделан
шаг 15 сделан
шаг 16 сделан
шаг 17 сделан
шаг 18 сделан
шаг 19 сделан
шаг 20 сделан
шаг 21 сделан
шаг 22 сделан
шаг 23 сделан
шаг 24 сделан
шаг 25 сделан
шаг 26 сделан
шаг 27 сделан
шаг 28 сделан
шаг 29 сделан
шаг 30 сделан
шаг 31 сделан
шаг 32 сделан
шаг 33 сделан
шаг 34 сделан
шаг 35 сделан
шаг 36 сделан
шаг 37 сделан
шаг 38 сделан
шаг 39 сделан
шаг 40 сделан
шаг 41 сделан
шаг 42 сделан
шаг 43 сделан
шаг 44 сделан
шаг 45 сделан
шаг 46 сделан
шаг 47 сделан
шаг 48 сделан
шаг 49 сделан
шаг 50 сделан
шаг 51 сделан
шаг 52 сделан
шаг 53 сделан
шаг 54 сделан
шаг 55 сделан
шаг 56 сделан
шаг 57 сделан
шаг 58 сделан
шаг 59 сделан
шаг 60 сделан
шаг 61 сделан
шаг 62 сделан
шаг 63 сделан
шаг 64 сделан
шаг 65 сделан
шаг 66 сделан
шаг 67 сделан
шаг 68 сделан
шаг 69 сделан
шаг 70 сделан
шаг 71 сделан
ша

### проверка

In [59]:
!ls -lht data

total 7784304
-rw-r--r--  1 da  staff   779M May  9 17:51 song_lists_preprocessed_tmp.txt
-rw-r--r--  1 da  staff   1.2G May  9 16:23 user_item_interaction_clear.csv
-rw-r--r--  1 da  staff   4.9M May  6 12:03 band_lists_preprocessed.csv
-rw-r--r--  1 da  staff    33M May  6 11:50 song_lists_preprocessed_small.txt
-rw-r--r--  1 da  staff    19M Apr 23 21:45 band_lists_preprocessed_small.txt
-rw-r--r--@ 1 da  staff   1.2G Apr 23 21:31 user_item_interaction.csv
-rw-r--r--@ 1 da  staff   464M Apr  4 17:40 track_meta_processed.tsv


Генератор для w2v, он считывает файл построчно, разделяет историю пользователя по \t на band_id и заменяет id на нормализованное название исполнителя

In [60]:
class TextToW2V:
    def __init__(self, file_path: str) -> None:
        """
        iterator for w2v out-of-memory training
        :param file_path: path to dataset in text format: one string for each user, only book IDs separated with ' '
        """
        self.file_path = file_path
        self.band_id_to_name = (pd.read_csv(meta_path, 
                                            sep='\t', usecols=['band_processed', 'song_id'],)
                                .set_index('song_id')
                                ['band_processed'].to_dict()
                               )

    def __iter__(self):
        """
        iterate over txt file and return user items
        :return:
        """
        for line in tqdm(open(self.file_path, 'r')):
            items = line.split('\t')
            yield [self.band_id_to_name.get(int(i_id), '') for i_id in items]

### дефолтное дообучение w2v, можно поробовать разные гиперпараметры
вспомните статью с их тюнигом: https://arxiv.org/pdf/1804.04212.pdf

In [61]:
corpus = TextToW2V(output_path)

model = word2vec.Word2Vec(corpus, min_count=20, sg=1, 
                          iter=1, window=10, 
                          workers=multiprocessing.cpu_count(),
                         )

print(f'Модель выучила {len(model.wv.vocab)} исполнителей.')

968756it [00:54, 17699.30it/s]
968756it [01:09, 13974.39it/s]

Модель выучила 27286 исполнителей.


# Smoke test модели

In [62]:
model.wv.most_similar('nirvana')

[('rhcp', 0.6815274953842163),
 ('foo fighters', 0.6738009452819824),
 ('bloodhound gang', 0.6731675863265991),
 ('p o d', 0.6728113889694214),
 ('jefferson airplane', 0.6686232089996338),
 ('iggy pop', 0.6673401594161987),
 ('саранча', 0.6638416647911072),
 ('chester bennington', 0.6629152297973633),
 ('guano apes', 0.6605006456375122),
 ('acdc', 0.6598184108734131)]